In [10]:
import os

import cv2 as cv
import numpy as np
from ultralytics import YOLO

model_path = os.path.join(os.getcwd(), "YOLO_model", "weights", "best.pt")
model = YOLO(model_path)

In [11]:
piece_names = {
    "white-king": "K",
    "white-queen": "Q",
    "white-rook": "R",
    "white-bishop": "B",
    "white-knight": "N",
    "white-pawn": "P",
    "black-king": "k",
    "black-queen": "q",
    "black-rook": "r",
    "black-bishop": "b",
    "black-knight": "n",
    "black-pawn": "p",
}

In [12]:
def find_pieces(board):
    pos = np.zeros((8, 8), dtype=np.dtype("U1"))
    res = model.predict(board)[0]
    coords = res.boxes.xyxy.numpy().astype(int)[:, 0:2]
    labels = [model.names[int(c)] for c in res.boxes.cls]
    board_height, board_width, _ = board.shape
    print(labels)
    print(res.boxes.conf)
    for i in range(len(coords)):
        x, y = coords[i]
        posX = round(8 * x / board_width)
        posY = round(8 * y / board_height)
        pos[posY, posX] = piece_names[labels[i]]

    return pos

In [14]:
img = cv.imread("scr.png")[:, :, :3]
img = cv.resize(img, (640, 640))
res = find_pieces(img)
from analysis import get_fen

print(get_fen(res, "w"))
print(res)


0: 640x640 1 black-king, 7 black-pawns, 1 black-queen, 1 black-rook, 2 white-bishops, 1 white-king, 6 white-pawns, 1 white-queen, 1 white-rook, 179.1ms
Speed: 2.1ms preprocess, 179.1ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)


['white-bishop', 'black-king', 'white-queen', 'black-queen', 'white-king', 'white-bishop', 'black-rook', 'black-pawn', 'black-pawn', 'white-rook', 'black-pawn', 'white-pawn', 'black-pawn', 'black-pawn', 'white-pawn', 'white-pawn', 'white-pawn', 'black-pawn', 'white-pawn', 'white-pawn', 'black-pawn']
tensor([0.9762, 0.9538, 0.9266, 0.9260, 0.9172, 0.9104, 0.9076, 0.8910, 0.8842, 0.8758, 0.8693, 0.8611, 0.8451, 0.8442, 0.8238, 0.8201, 0.8168, 0.8145, 0.7954, 0.7246, 0.2523])
5rk1/1p2q1pp/p3R3/3p1p2/Q2B4/2P4B/PP3PPP/6K1 w
[['' '' '' '' '' 'r' 'k' '']
 ['' 'p' '' '' 'q' '' 'p' 'p']
 ['p' '' '' '' 'R' '' '' '']
 ['' '' '' 'p' '' 'p' '' '']
 ['Q' '' '' 'B' '' '' '' '']
 ['' '' 'P' '' '' '' '' 'B']
 ['P' 'P' '' '' '' 'P' 'P' 'P']
 ['' '' '' '' '' '' 'K' '']]
